In [6]:
import requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import pandas as pd
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import os
import numpy as np

import pymysql
import sqlalchemy as alch
from getpass import getpass

#import googletrans
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#import spacy

from dotenv import load_dotenv
import re

In [107]:
def get_all_restaurant_data(start, end):
    all_restaurant_data = []
    for offset in range(start, end, 30):
        #Every page shows 30 restaurants
        url = f'https://www.tripadvisor.com/RestaurantSearch-g187497-oa{offset}-Barcelona_Catalonia.html#EATERY_LIST_CONTENTS'
        # As tripadvisor recognized python and blocked it I had to use header User-Agent
        html = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})
        soup = BeautifulSoup(html.content, "html.parser")
        
        tags_name = soup.find_all("a", class_="Lwqic Cj b")
        tags_website = [a.get('href') for a in soup.find_all("a", class_="Lwqic Cj b", href=True)]
        amount_reviews = soup.find_all("span", class_="IiChw")
        tags_rating = soup.find_all("svg", class_="UctUV d H0")
        cuisine_type = soup.find_all("svg", class_="SUszq")

        names = [i.getText().strip() for i in tags_name]
        # As it is a ranking, the output is (rank. Name of restaurant), so we remove the numbers
        names = [re.sub(r'^[^.]*\.', '', name) for name in names]
        website = [i for i in tags_website]    
        reviews = [re.sub(r'\D', '', i.getText().strip()) for i in amount_reviews]
        ratings = [tag.get('aria-label')[:3] for tag in tags_rating]
    

    
        # Zip to add all values in a tuple for each restaurant
        rows = list(zip(names, website, reviews, ratings))
        all_restaurant_data.extend(rows)
        
    df_ta = pd.DataFrame(all_restaurant_data, columns=['Name', 'Website', 'Reviews', 'Rating'])
    return df_ta



In [206]:
df_restaurants = get_all_restaurant_data(60,190)
df_restaurants.to_csv('df_restaurants.csv', index = True)

In [131]:
import numpy as np

def get_restaurant_data(country, start, end):
    # Define df_restaurants inside the function
    df_restaurants = pd.read_csv('df_restaurants.csv')

    restaurant_data = []
    for i in range(len(df_restaurants)):
        url = 'https://www.tripadvisor.{}/{}'.format(country, df_restaurants['Website'][i])
        for offset in range(start, end, 10):
            url = url.replace('Reviews', 'Reviews-or{}'.format(offset))
            html = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})
            soup = BeautifulSoup(html.content, "html.parser")

            # Find the review text and rating
            tag_review = soup.find_all("div", class_="review-container")
            review = [i.find("p", class_="partial_entry").getText().strip() for i in tag_review]
            rating = [int(i.find("span", class_=re.compile("bubble_\d+"))["class"][1][7:]) for i in tag_review]
            restaurant_name_tag = soup.find('h1', {'data-test-target': 'top-info-header'})
            if restaurant_name_tag is not None:
                restaurant_name = restaurant_name_tag.text
            else:
                restaurant_name = np.nan

            # Find the date of visit
            tag_date = soup.find_all("div", class_="prw_rup prw_reviews_stay_date_hsx")
            date = []
            for i in tag_date:
                date_span = i.find("span", class_="stay_date_label")
                if date_span:
                    date_value = date_span.next_sibling.strip()
                else:
                    date_value = np.nan
                date.append(date_value)

            # Extract language and restaurant information
            language = country
            #restaurant = df_restaurants['Name'][i]

            # Zip to add all values in a tuple for each restaurant
            rows = list(zip(review, rating, date, [language]*len(review), [restaurant_name]*len(review)))
            restaurant_data.extend(rows)

    df_ta = pd.DataFrame(restaurant_data, columns=['review', 'rating', 'date', 'language', 'restaurant'])
    return df_ta


In [207]:
df_de = (get_restaurant_data('DE', 0, 15))
df_de.to_csv('df_de.csv', index = False)
df_de.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)


In [208]:

df_nl = (get_restaurant_data('NL', 0, 15))
df_nl.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_nl.to_csv('df_nl.csv', index = False)


ConnectionError: HTTPSConnectionPool(host='www.tripadvisor.nl', port=443): Max retries exceeded with url: //Restaurant_Review-g187497-d11881068-Reviews-or10-or0-Teoric_Taverna_Gastronomica-Barcelona_Catalonia.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002077AE0D480>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

,review,rating,date,language,restaurant
0,"Een heel gezellig restaurantje, met een heel g...",50,februari 2023,NL,La Gastronomica Burgers
1,Zalige burgers en een zeer leuke en vlotte bed...,50,februari 2023,NL,La Gastronomica Burgers
2,Heel klein restaurantje met alleen hamburgers ...,50,oktober 2022,NL,La Gastronomica Burgers
3,Klein restaurant met enthousiaste eigenaar en ...,50,oktober 2022,NL,La Gastronomica Burgers
4,Hierheen gegaan net als alle Tripadvisor foodi...,50,september 2022,NL,La Gastronomica Burgers
...,...,...,...,...,...
480,Top adres lekkere tapas vriendelijke mensen we...,50,oktober 2022,NL,Bodega Oliva
481,"Klein, maar súper knus. Zò ontzettend gastvrij...",50,juli 2022,NL,Bodega Oliva
482,"Hele lekkere tapas, goeie wijnen en aardige en...",50,juni 2022,NL,Bodega Oliva
483,Klein maar erg fijn. Gastvrouw en kok snappen ...,50,juni 2022,NL,Bodega Oliva


In [ ]:
df_es = (get_restaurant_data('ES', 0, 15))
df_es.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_es.to_csv('df_es.csv', index = False)

,review,rating,date,language,restaurant
0,Riquísimas burguers y como en casa!\n\nMuy bue...,50,febrero de 2023,ES,La Gastronómica Burgers
1,"Las hamburguesas brutales, buenos precios y la...",50,febrero de 2023,ES,La Gastronómica Burgers
2,"Hamburguesas muy ricas, especialmente la de Co...",50,febrero de 2023,ES,La Gastronómica Burgers
3,Gracias Joel por el trato q nos has dado! Hemo...,50,febrero de 2023,ES,La Gastronómica Burgers
4,Íbamos sin rumbo con unas amigas buscando dond...,50,febrero de 2023,ES,La Gastronómica Burgers
...,...,...,...,...,...
567,"Fuimos hace algún tiempo, y siempre quisimos r...",50,abril de 2022,ES,Anema E Core
568,Me serví una copa de sangría con una cucaracha...,10,marzo de 2022,ES,Anema E Core
569,De los mejores restaurantes de bsrcelona donde...,50,enero de 2022,ES,Anema E Core
570,Buena calidad de tapas con muy buen ambiente💃 ...,50,septiembre de 2022,ES,Bodega Oliva


In [ ]:
df_it = (get_restaurant_data('IT', 0, 15))
df_it.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_it.to_csv('df_it.csv', index = False)

In [ ]:
df_fr = (get_restaurant_data('FR', 0, 15))
df_fr.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_fr.to_csv('df_fr.csv', index = False)


In [ ]:
df_pt = (get_restaurant_data('PT', 0, 15))
df_pt.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_pt.to_csv('df_pt.csv', index = False)

c:\Users\Tim_K\miniconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [ ]:
df_se = (get_restaurant_data('SE', 0, 15))
df_se.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_se.to_csv('df_se.csv', index = False)

In [ ]:
#df_no = (get_restaurant_data('NO', 0, 15))
#df_no.drop_duplicates(subset=['review'])
#df_no.to_csv('df_no.csv', index = False)

In [ ]:
#df_fi = (get_restaurant_data('FI', 0, 15))
#df_fi.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
#df_fi.to_csv('df_fi.csv', index = False)

In [133]:
df_fi

,review,rating,date,language,restaurant
0,Had a wonderful meal here! The best bit was th...,50,February 2023,FI,La Gastronomica Burgers
1,Incredible burgers. The different kinds of mea...,50,February 2023,FI,La Gastronomica Burgers
2,Nice burgers in a small setting (roughly a doz...,40,February 2023,FI,La Gastronomica Burgers
3,I'we had lot of burgers in my lifetime (and ha...,30,February 2023,FI,La Gastronomica Burgers
4,Good option if you are looking for a quality h...,50,February 2023,FI,La Gastronomica Burgers
...,...,...,...,...,...
872,Everything about this place is amazing! Defini...,50,January 2023,FI,Bodega Oliva
873,What a lovely evening. My wife and I went for ...,50,January 2023,FI,Bodega Oliva
874,Just a perfect evening! Everythnig was very go...,50,December 2022,FI,Bodega Oliva
875,"Wow, what can I say. An absolute wonderful ex...",50,December 2022,FI,Bodega Oliva


In [ ]:
df_gr = (get_restaurant_data('COM.GR', 0, 15))
df_gr.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_gr.to_csv('df_gr.csv', index = False)

In [ ]:
#df_cn = (get_restaurant_data('CN', 0, 15))
#df_cn.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
#df_cn.to_csv('df_cn.csv', index = False)

ConnectTimeout: HTTPSConnectionPool(host='www.tripadvisor.cn', port=443): Max retries exceeded with url: //Restaurant_Review-g187497-d25012128-Reviews-or10-or0-Xera_Restaurant-Barcelona_Catalonia.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000020778787280>, 'Connection to www.tripadvisor.cn timed out. (connect timeout=None)'))

In [ ]:
df_tr = (get_restaurant_data('COM.TR', 0, 15))
df_tr.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_tr.to_csv('df_tr.csv', index = False)


In [ ]:
df_ru = (get_restaurant_data('RU', 0, 15))
df_ru.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_ru.to_csv('df_ru.csv', index = False)

In [ ]:
df_en = (get_restaurant_data('COM', 0, 15))
df_en.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_en.to_csv('df_en.csv', index = False)

In [ ]:
df_jp = (get_restaurant_data('JP', 0, 15))
df_jp.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_jp.to_csv('df_jp.csv', index = False)

In [ ]:
df_il = (get_restaurant_data('CO.IL', 0, 15))
df_il.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_il.to_csv('df_il.csv', index = False)

In [ ]:
df_vn = (get_restaurant_data('COM.VN', 0, 15))
df_vn.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_vn.to_csv('df_vn.csv', index = False)

In [ ]:
df_kr = (get_restaurant_data('CO.KR', 0, 15))
df_kr.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_kr.to_csv('df_kr.csv', index = False)

In [ ]:
df_in = (get_restaurant_data('IN', 0, 15))
df_in.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_in.to_csv('df_in.csv', index = False)

In [160]:
#df_all_languages.to_csv('df_all_languages.csv', index = True)

In [185]:
import numpy as np

def get_restaurant_data2(country, start, end):
    # Define df_restaurants inside the function
    df_restaurants = pd.read_csv('df_restaurants.csv')

    restaurant_data = []
    for i in range(len(df_restaurants)):
        url = 'https://{}.tripadvisor.com/{}'.format(country, df_restaurants['Website'][i])
        for offset in range(start, end, 10):
            url = url.replace('Reviews', 'Reviews-or{}'.format(offset))
            html = requests.get(url, headers={'User-Agent': "Mozilla/5.0"})
            soup = BeautifulSoup(html.content, "html.parser")

            # Find the review text and rating
            tag_review = soup.find_all("div", class_="review-container")
            review = [i.find("p", class_="partial_entry").getText().strip() for i in tag_review]
            rating = [int(i.find("span", class_=re.compile("bubble_\d+"))["class"][1][7:]) for i in tag_review]
            restaurant_name_tag = soup.find('h1', {'data-test-target': 'top-info-header'})
            if restaurant_name_tag is not None:
                restaurant_name = restaurant_name_tag.text
            else:
                restaurant_name = np.nan

            # Find the date of visit
            tag_date = soup.find_all("div", class_="prw_rup prw_reviews_stay_date_hsx")
            date = []
            for i in tag_date:
                date_span = i.find("span", class_="stay_date_label")
                if date_span:
                    date_value = date_span.next_sibling.strip()
                else:
                    date_value = np.nan
                date.append(date_value)

            # Extract language and restaurant information
            language = country
            #restaurant = df_restaurants['Name'][i]

            # Zip to add all values in a tuple for each restaurant
            rows = list(zip(review, rating, date, [language]*len(review), [restaurant_name]*len(review)))
            restaurant_data.extend(rows)

    df_ta = pd.DataFrame(restaurant_data, columns=['review', 'rating', 'date', 'language', 'restaurant'])
    return df_ta


In [ ]:
df_no = (get_restaurant_data2('NO', 0, 15))
df_no.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_no.to_csv('df_no.csv', index = False)

In [ ]:
df_ar = (get_restaurant_data2('ar', 0, 15))
df_ar.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_ar.to_csv('df_ar.csv', index = False)

In [ ]:
df_th = (get_restaurant_data2('TH', 0, 15))
df_th.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_th.to_csv('df_th.csv', index = False)

In [ ]:
df_cn = (get_restaurant_data2('CN', 0, 15))
df_cn.drop_duplicates(subset=['review', 'date'], keep='first', inplace=True)
df_cn.to_csv('df_cn.csv', index = False)

In [216]:
df_all_languages = pd.concat([df_all_languages3, df_de, df_es, df_fr, df_it, df_nl, df_pt, df_se, df_gr, df_tr, df_ru, df_en, df_jp, df_in, df_kr, df_vn, df_il, df_cn, df_th, df_ar, df_no], axis=0, ignore_index=True)


In [213]:
df_all_languages.to_csv('df_all_languages.csv', index = True)

In [217]:
df_all_languages

,review,rating,date,language,restaurant
0,Muss man mal gegessen haben. Die Burger sind f...,50,November 2022,DE,La Gastronomica Burgers
1,Traumhaft! Höchste Qualität und ein ganz tolle...,50,November 2022,DE,La Gastronomica Burgers
2,Die Burger haben alle unsere Erwartungen übert...,50,November 2022,DE,La Gastronomica Burgers
3,Amazing!! Einer der besten Burger die ich je h...,50,Oktober 2022,DE,La Gastronomica Burgers
4,Sehr leckere Burger zu erschwinglichen Preisen...,50,Oktober 2022,DE,La Gastronomica Burgers
...,...,...,...,...,...
22910,Andre gang i Barcelona og kanskje 5 - 6 i denn...,50,april 2019,NO,Prado de Flores
22911,Maten er veldig bra. Tjenesten er også veldig ...,50,oktober 2018,NO,Prado de Flores
22912,Var ganske nær det viktigste turistområdet så ...,40,august 2018,NO,Prado de Flores
22913,"God service, god mat og øl:). . . . . alle anb...",50,oktober 2018,NO,Prado de Flores


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

url = "https://www.tripadvisor.com/Restaurant_Review-g187497-d12702677-Reviews-La_Gastronomica_Burgers-Barcelona_Catalonia.html"
driver = webdriver.Chrome()
driver.get(url)

# Wait for the "I Accept" button to load
WebDriverWait(driver, 90).until(
    EC.presence_of_element_located((By.ID, "onetrust-button-group"))
)

# Click the "I Accept" button
accept_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
accept_button.click()

# Click the "More languages" button
more_languages = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, "//div[@class='taLnk' and span[text()='More languages']]"))
)

# Wait for 5 seconds
time.sleep(5)

# Click the "More languages" element
more_languages.click()

# Wait for the "Dutch" button element to be clickable
dutch_language = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='filters_detail_language_filterLang_nl']"))
)

# Click the "Dutch" button element
dutch_language.click()

html = ''
while True:
    try:
        # Wait for the "Next" button element to be clickable
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".pagination .nav.next"))
        )
    except:
        break

    # Click the next button
    next_button.click()

    # Wait for the reviews to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".review-container"))
    )

    # Get the HTML content of the current page
    html += driver.execute_script("return document.documentElement.outerHTML")

    # Check if the next button is disabled
    if "disabled" in next_button.get_attribute("class"):
        break

soup = BeautifulSoup(html, "html.parser")
feed = soup.find_all("div", {"class":"ratings_and_types block_wrap ui_section"})[0]
posts = feed.find_all("div", {"class":"ui_column is-9"})


IndexError: list index out of range

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

html = []

url = "https://www.tripadvisor.com/Restaurant_Review-g187497-d12702677-Reviews-La_Gastronomica_Burgers-Barcelona_Catalonia.html"
driver = webdriver.Chrome()
driver.get(url)

# Wait for the "I Accept" button to load
WebDriverWait(driver, 90).until(
    EC.presence_of_element_located((By.ID, "onetrust-button-group"))
)

# Click the "I Accept" button
accept_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
accept_button.click()

# Click the "More languages" button
more_languages = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, "//div[@class='taLnk' and span[text()='More languages']]"))
)

# Wait for 5 seconds
time.sleep(5)

# Click the "More languages" element
more_languages.click()

# Wait for the "Dutch" button element to be clickable
dutch_language = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='filters_detail_language_filterLang_nl']"))
)

# Click the "Dutch" button element
dutch_language.click()

while True:
    # Wait for the "Next" button element to be clickable
    next_button = None
    for i in range(3):
        try:
            next_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'nav.next'))
            )
            break
        except:
            time.sleep(3)
            
    if next_button is None:
        break
        
    # Append the current page HTML to the list
    html.append(driver.execute_script("return document.documentElement.outerHTML;"))
    
    # Click the next button
    for i in range(3):
        try:
            next_button.click()
            break
        except:
            time.sleep(3)
            continue
        
soup = BeautifulSoup(''.join(html), "html.parser")
feed = soup.find_all("div", {"class":"ratings_and_types block_wrap ui_section"})[0]
posts = feed.find_all("div", {"class":"ui_column is-9"})

with open("reviews.html", "w", encoding="utf-8") as f:
    f.write(str(soup))


KeyboardInterrupt: 

In [37]:
import pandas as pd
import re

Reviews_Dutch = pd.DataFrame(columns=['Rating', 'Date', 'Restaurant', 'Review_1'])

for post in posts:
    # Extract the Rating
    rating = re.search(r'bubble_(\d{2})', str(post)).group(1)
    
    # Extract the Date
    date = re.search(r'title="(.+?)"', str(post)).group(1)
    
    # Extract the Restaurant
    restaurant = re.search(r'href="/(.+?)"', str(post)).group(1)

    review = re.search(r'partial_entry">(.*?)"', str(post))
    
    # Define the regex pattern to search for the review text
    #pattern = r'partial_entry">(.*?)</span></p></div></div>'

    # Search for the review text using the regex pattern and save it to a new column in the dataframe
    #Reviews_Dutch['Review_1'] = Reviews_Dutch['Review_1'].apply(lambda x: re.search(pattern, x, re.DOTALL).group(1))

    
    Reviews_Dutch = Reviews_Dutch.append({
        'Rating': rating,
        'Date': date,
        'Restaurant': restaurant,
        'Review': review,  
    }, ignore_index=True)


C:\Users\Tim_K\AppData\Local\Temp\ipykernel_11784\2510392009.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Reviews_Dutch = Reviews_Dutch.append({
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_11784\2510392009.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Reviews_Dutch = Reviews_Dutch.append({
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_11784\2510392009.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Reviews_Dutch = Reviews_Dutch.append({
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_11784\2510392009.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Reviews_Dutch = Reviews_Dutch.append({
C:\Users\Tim_K\AppData\Local\Tem

In [38]:
Reviews_Dutch 

,Rating,Date,Restaurant,Review_1,Review
0,50,"August 23, 2019",ShowUserReviews-g187497-d12702677-r702255865-L...,NaN,None
1,50,"June 13, 2019",ShowUserReviews-g187497-d12702677-r681209861-L...,NaN,"<re.Match object; span=(1039, 1202), match='pa..."
2,50,"June 5, 2019",ShowUserReviews-g187497-d12702677-r679288926-L...,NaN,"<re.Match object; span=(1050, 1196), match='pa..."
3,50,"May 14, 2019",ShowUserReviews-g187497-d12702677-r674081201-L...,NaN,"<re.Match object; span=(1193, 1328), match='pa..."
4,50,"March 23, 2019",ShowUserReviews-g187497-d12702677-r660584032-L...,NaN,None
5,50,"February 2, 2019",ShowUserReviews-g187497-d12702677-r649643958-L...,NaN,"<re.Match object; span=(1200, 1343), match='pa..."
6,50,"January 25, 2019",ShowUserReviews-g187497-d12702677-r648201313-L...,NaN,"<re.Match object; span=(1195, 1344), match='pa..."
7,50,"August 27, 2018",ShowUserReviews-g187497-d12702677-r610822696-L...,NaN,"<re.Match object; span=(1195, 1345), match='pa..."
8,50,"July 12, 2018",ShowUserReviews-g187497-d12702677-r595680805-L...,NaN,None
9,50,"June 12, 2018",ShowUserReviews-g187497-d12702677-r587014463-L...,NaN,None


In [ ]:
posts[0]

: 

: 